
# Single bead evaluation plot

This example shows how to use PyAdditive-Widgets to visualize the results
of a parametric study containing single bead simulations.
It uses the :obj:`display <ansys.additive.widgets.display>` package to
visualize the results of the study.

Units are SI (m, kg, s, K) unless otherwise noted.


## Perform required imports and create study
Perform the required imports and create a :class:`ParametricStudy` instance.



In [ ]:
from ansys.additive.core import Additive, SimulationStatus, SimulationType
from ansys.additive.core.parametric_study import ColumnNames, ParametricStudy

from ansys.additive.widgets import display

study = ParametricStudy("single-bead-study")

## Get name of study file
The current state of the parametric study is saved to a file on each
update. The following code retrieves the name of the file. This file
uses a binary format and is not human readable.



In [ ]:
print(study.file_name)

## Select material for study
Select a material to use in the study. The material name must be known by
the Additive service. You can connect to the Additive service
and print a list of available materials prior to selecting one.



In [ ]:
additive = Additive()
additive.materials_list()
material = "IN718"

## Create single bead evaluation
Generate single bead simulation permutations using the
:meth:`~ParametricStudy.generate_single_bead_permutations` method. This method's
parameters allow you to specify a range of machine parameters and filter them by
energy density. Not all the parameters shown are required. Optional parameters
that are not specified use the default values defined in the
:class:`MachineConstants` class.



In [ ]:
import numpy as np

# Specify a range of laser powers. Valid values are 50 to 700 W.
initial_powers = np.linspace(50, 700, 7)
# Specify a range of laser scan speeds. Valid values are 0.35 to 2.5 m/s.
initial_scan_speeds = np.linspace(0.35, 2.5, 5)
# Specify powder layer thicknesses. Valid values are 10e-6 to 100e-6 m.
initial_layer_thicknesses = [40e-6, 50e-6]
# Specify laser beam diameters. Valid values are 20e-6 to 140e-6 m.
initial_beam_diameters = [80e-6]
# Specify heater temperatures. Valid values are 20 - 500 C.
initial_heater_temps = [80]
# Restrict the permutations within a range of energy densities
# For a single bead, the energy density is laser power / (laser scan speed * layer thickness).
min_energy_density = 2e6
max_energy_density = 8e6
# Specify a bead length in meters.
bead_length = 0.001

study.generate_single_bead_permutations(
    material_name=material,
    bead_length=bead_length,
    laser_powers=initial_powers,
    scan_speeds=initial_scan_speeds,
    layer_thicknesses=initial_layer_thicknesses,
    beam_diameters=initial_beam_diameters,
    heater_temperatures=initial_heater_temps,
    min_area_energy_density=min_energy_density,
    max_area_energy_density=max_energy_density,
)

## Show simulations as a table
Use the :obj:`display <ansys.additive.widgets.display>`
package to list the simulations as a table.



In [ ]:
display.show_table(study)

## Skip some simulations
If you are working with a large parametric study, you might want to skip some
simulations to reduce processing time. To do so, set the simulation status,
which is defined in the :class:`SimulationStatus` class, to :obj:`SimulationStatus.SKIP`.
The following code obtains a :class:`~pandas.DataFrame` object, applies a filter to
get a list of simulation IDs, and then updates the status on the simulations
with those IDs.



In [ ]:
df = study.data_frame()
# Get IDs for single bead simulations with laser power below 75 W.
ids = df.loc[
    (df[ColumnNames.LASER_POWER] < 75) & (df[ColumnNames.TYPE] == SimulationType.SINGLE_BEAD),
    ColumnNames.ID,
].tolist()
study.set_status(ids, SimulationStatus.SKIP)
display.show_table(study)

## Run single bead simulations
Run the simulations using the :meth:`~ParametricStudy.run_simulations` method.
All simulations with a :obj:`SimulationStatus.PENDING` status are executed.



In [ ]:
study.run_simulations(additive)

## Plot single bead results
Plot the single bead results using the
:func:`~ansys.additive.widgets.display.single_bead_eval_plot` function.



In [ ]:
display.single_bead_eval_plot(study)